In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

Vanishing/Exploding Gradients problem

In [2]:
def logit(z):
    return 1/(1+np.exp(-z))

Nonsaturating Activation Functions

In [3]:
#Leaky ReLU
def leaky_relu(z,alpha=0.01):
    return np.maximum(alpha*z,z)

Train the Fashin MNIST using Leaky ReLU

In [4]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:6719]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:6719]

In [5]:
X_train.shape

(1719, 28, 28)

In [6]:
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.Dense(300,kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100,kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10,activation="softmax")
])

In [7]:
model.compile(loss="sparse_categorical_crossentropy",
             Optimizer=keras.optimizers.SGD(lr=1e-3),
             metrics=["accuracy"])

In [8]:
history = model.fit(X_train,y_train,epochs=10,
                   validation_data=(X_valid,y_valid))

Train on 1719 samples, validate on 5000 samples
Epoch 1/10
1719/1719 [==============================] - 1s 858us/sample - loss: 1.1421 - accuracy: 0.6056 - val_loss: 0.8728 - val_accuracy: 0.6604
Epoch 2/10
1719/1719 [==============================] - 1s 398us/sample - loss: 0.7209 - accuracy: 0.7283 - val_loss: 0.7683 - val_accuracy: 0.7134
Epoch 3/10
1719/1719 [==============================] - 1s 393us/sample - loss: 0.6116 - accuracy: 0.7818 - val_loss: 0.5685 - val_accuracy: 0.8022
Epoch 4/10
1719/1719 [==============================] - 1s 396us/sample - loss: 0.5227 - accuracy: 0.8034 - val_loss: 0.7273 - val_accuracy: 0.7448
Epoch 5/10
1719/1719 [==============================] - 1s 467us/sample - loss: 0.5114 - accuracy: 0.8010 - val_loss: 0.5974 - val_accuracy: 0.7832
Epoch 6/10
1719/1719 [==============================] - 1s 402us/sample - loss: 0.4403 - accuracy: 0.8348 - val_loss: 0.5707 - val_accuracy: 0.8018
Epoch 7/10
1719/1719 [==============================] - 1s 397us

In [9]:
#PReLU
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [10]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [11]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Train on 1719 samples, validate on 5000 samples
Epoch 1/10
1719/1719 [==============================] - 1s 743us/sample - loss: 2.2819 - accuracy: 0.1547 - val_loss: 2.1987 - val_accuracy: 0.1952
Epoch 2/10
1719/1719 [==============================] - 1s 384us/sample - loss: 2.1062 - accuracy: 0.2792 - val_loss: 2.0555 - val_accuracy: 0.3150
Epoch 3/10
1719/1719 [==============================] - 1s 381us/sample - loss: 1.9840 - accuracy: 0.3950 - val_loss: 1.9453 - val_accuracy: 0.4060
Epoch 4/10
1719/1719 [==============================] - 1s 382us/sample - loss: 1.8856 - accuracy: 0.4729 - val_loss: 1.8529 - val_accuracy: 0.4830
Epoch 5/10
1719/1719 [==============================] - 1s 378us/sample - loss: 1.8015 - accuracy: 0.5323 - val_loss: 1.7728 - val_accuracy: 0.5344
Epoch 6/10
1719/1719 [==============================] - 1s 384us/sample - loss: 1.7267 - accuracy: 0.5753 - val_loss: 1.7010 - val_accuracy: 0.5694
Epoch 7/10
1719/1719 [==============================] - 1s 390us

In [12]:
#ELU
def elu(z, alpha=1):
    return np.where(z < 0, alpha * (np.exp(z) - 1), z)

SELU: During training, a neural network composed exclusively of a stack of dense layers using the SELU activation function and LeCun initialization will self-normalize: the output of each layer will tend to preserve the same mean and variance during training, which solves the vanishing/exploding gradients problem.

In [13]:
from scipy.special import erfc

# alpha and scale to self normalize with mean 0 and standard deviation 1
# (see equation 14 in the paper):
alpha_0_1 = -np.sqrt(2 / np.pi) / (erfc(1/np.sqrt(2)) * np.exp(1/2) - 1)
scale_0_1 = (1 - erfc(1 / np.sqrt(2)) * np.sqrt(np.e)) * np.sqrt(2 * np.pi) * (2 * erfc(np.sqrt(2))*np.e**2 + np.pi*erfc(1/np.sqrt(2))**2*np.e - 2*(2+np.pi)*erfc(1/np.sqrt(2))*np.sqrt(np.e)+np.pi+2)**(-1/2)

In [14]:
def selu(z, scale=scale_0_1, alpha=alpha_0_1):
    return scale * elu(z, alpha)

In [15]:
#SELU,for Dense layers
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(300,activation="selu",kernel_initializer="lecun_normal"))

for layer in range(99):
    model.add(keras.layers.Dense(100,activation="selu",kernel_initializer="lecun_normal"))
    model.add(keras.layers.Dense(10,activation="softmax"))

In [16]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

Now, train it. But It has to be scaled the inputs to mean 0 and SD 1

In [17]:
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

In [18]:
history = model.fit(X_train_scaled, y_train, epochs=5,
                    validation_data=(X_valid_scaled, y_valid))

Train on 1719 samples, validate on 5000 samples
Epoch 1/5
1719/1719 [==============================] - 27s 16ms/sample - loss: 2.3160 - accuracy: 0.1041 - val_loss: 2.3166 - val_accuracy: 0.0980
Epoch 2/5
1719/1719 [==============================] - 11s 7ms/sample - loss: 2.3150 - accuracy: 0.1041 - val_loss: 2.3156 - val_accuracy: 0.0980
Epoch 3/5
1719/1719 [==============================] - 11s 7ms/sample - loss: 2.3140 - accuracy: 0.1041 - val_loss: 2.3147 - val_accuracy: 0.0980
Epoch 4/5
1719/1719 [==============================] - 11s 7ms/sample - loss: 2.3131 - accuracy: 0.1041 - val_loss: 2.3139 - val_accuracy: 0.0980
Epoch 5/5
1719/1719 [==============================] - 11s 7ms/sample - loss: 2.3123 - accuracy: 0.1041 - val_loss: 2.3132 - val_accuracy: 0.0980


# Batch Normalization

In [19]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300,activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300,activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10,activation="softmax")
])

In [20]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense_205 (Dense)            (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_206 (Dense)            (None, 300)               90300     
_________________________________________________________________
batch_normalization_2 (Batch (None, 300)               1200      
_________________________________________________________________
dense_207 (Dense)            (None, 10)               

In [21]:
bn1 = model.layers[1]
[(var.name,var.trainable) for var in bn1.variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

3rd and 4th values are not trainable

In [22]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [23]:
history = model.fit(X_train, y_train, epochs=15,
                    validation_data=(X_valid, y_valid))

Train on 1719 samples, validate on 5000 samples
Epoch 1/15
1719/1719 [==============================] - 2s 1ms/sample - loss: 2.3279 - accuracy: 0.2490 - val_loss: 2.0339 - val_accuracy: 0.3708
Epoch 2/15
1719/1719 [==============================] - 1s 707us/sample - loss: 1.3640 - accuracy: 0.5631 - val_loss: 1.6730 - val_accuracy: 0.5938
Epoch 3/15
1719/1719 [==============================] - 1s 697us/sample - loss: 1.0904 - accuracy: 0.6603 - val_loss: 1.3316 - val_accuracy: 0.6598
Epoch 4/15
1719/1719 [==============================] - 1s 698us/sample - loss: 0.9618 - accuracy: 0.6800 - val_loss: 1.0783 - val_accuracy: 0.7012
Epoch 5/15
1719/1719 [==============================] - 1s 701us/sample - loss: 0.8754 - accuracy: 0.7109 - val_loss: 0.9222 - val_accuracy: 0.7210
Epoch 6/15
1719/1719 [==============================] - 1s 755us/sample - loss: 0.8251 - accuracy: 0.7248 - val_loss: 0.8314 - val_accuracy: 0.7318
Epoch 7/15
1719/1719 [==============================] - 1s 765us/s

Sometimes applying BN before the activation function works better

In [24]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(100, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [25]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [26]:
history = model.fit(X_train, y_train, epochs=15,
                    validation_data=(X_valid, y_valid))

Train on 1719 samples, validate on 5000 samples
Epoch 1/15
1719/1719 [==============================] - 2s 1ms/sample - loss: 2.3968 - accuracy: 0.1652 - val_loss: 2.1777 - val_accuracy: 0.2450
Epoch 2/15
1719/1719 [==============================] - 1s 594us/sample - loss: 1.9603 - accuracy: 0.3333 - val_loss: 1.9622 - val_accuracy: 0.3922
Epoch 3/15
1719/1719 [==============================] - 1s 597us/sample - loss: 1.6486 - accuracy: 0.4887 - val_loss: 1.7401 - val_accuracy: 0.5332
Epoch 4/15
1719/1719 [==============================] - 1s 586us/sample - loss: 1.4638 - accuracy: 0.5730 - val_loss: 1.5315 - val_accuracy: 0.6136
Epoch 5/15
1719/1719 [==============================] - 1s 567us/sample - loss: 1.3451 - accuracy: 0.6184 - val_loss: 1.3603 - val_accuracy: 0.6472
Epoch 6/15
1719/1719 [==============================] - 1s 599us/sample - loss: 1.2410 - accuracy: 0.6486 - val_loss: 1.2308 - val_accuracy: 0.6672
Epoch 7/15
1719/1719 [==============================] - 1s 611us/s

# Reusing Pretrained Layers

In [27]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [28]:
X_train_A.shape

(1378, 28, 28)

In [29]:
X_train_B.shape

(200, 28, 28)

In [30]:
tf.random.set_seed(42)
np.random.seed(42)

In [31]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [32]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=["accuracy"])

In [33]:
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A))

Train on 1378 samples, validate on 4014 samples
Epoch 1/20
1378/1378 [==============================] - 2s 1ms/sample - loss: 2.0484 - accuracy: 0.2409 - val_loss: 1.5895 - val_accuracy: 0.4096
Epoch 2/20
1378/1378 [==============================] - 1s 681us/sample - loss: 1.3521 - accuracy: 0.5668 - val_loss: 1.1762 - val_accuracy: 0.6450
Epoch 3/20
1378/1378 [==============================] - 1s 639us/sample - loss: 1.0435 - accuracy: 0.7177 - val_loss: 0.9578 - val_accuracy: 0.7282
Epoch 4/20
1378/1378 [==============================] - 1s 627us/sample - loss: 0.8712 - accuracy: 0.7649 - val_loss: 0.8160 - val_accuracy: 0.7775
Epoch 5/20
1378/1378 [==============================] - 1s 613us/sample - loss: 0.7615 - accuracy: 0.7990 - val_loss: 0.7361 - val_accuracy: 0.7910
Epoch 6/20
1378/1378 [==============================] - 1s 620us/sample - loss: 0.6866 - accuracy: 0.8048 - val_loss: 0.6925 - val_accuracy: 0.7917
Epoch 7/20
1378/1378 [==============================] - 1s 627us/s

In [34]:
model_A.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_211 (Dense)            (None, 300)               235500    
_________________________________________________________________
dense_212 (Dense)            (None, 100)               30100     
_________________________________________________________________
dense_213 (Dense)            (None, 50)                5050      
_________________________________________________________________
dense_214 (Dense)            (None, 50)                2550      
_________________________________________________________________
dense_215 (Dense)            (None, 50)                2550      
_________________________________________________________________
dense_216 (Dense)            (None, 8)                

In [35]:
model_A.save("my_model_A.h5")

In [36]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [37]:
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=["accuracy"])

In [38]:
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

Train on 200 samples, validate on 986 samples
Epoch 1/20
200/200 [==============================] - 1s 5ms/sample - loss: 0.9545 - accuracy: 0.4600 - val_loss: 0.6655 - val_accuracy: 0.5385
Epoch 2/20
200/200 [==============================] - 0s 1ms/sample - loss: 0.5899 - accuracy: 0.6900 - val_loss: 0.4785 - val_accuracy: 0.8519
Epoch 3/20
200/200 [==============================] - 0s 1ms/sample - loss: 0.4512 - accuracy: 0.8800 - val_loss: 0.4098 - val_accuracy: 0.8945
Epoch 4/20
200/200 [==============================] - 0s 1ms/sample - loss: 0.3871 - accuracy: 0.9100 - val_loss: 0.3666 - val_accuracy: 0.9128
Epoch 5/20
200/200 [==============================] - 0s 1ms/sample - loss: 0.3438 - accuracy: 0.9250 - val_loss: 0.3315 - val_accuracy: 0.9300
Epoch 6/20
200/200 [==============================] - 0s 1ms/sample - loss: 0.3095 - accuracy: 0.9300 - val_loss: 0.3034 - val_accuracy: 0.9402
Epoch 7/20
200/200 [==============================] - 0s 1ms/sample - loss: 0.2810 - accur

In [39]:
model_B.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_217 (Dense)            (None, 300)               235500    
_________________________________________________________________
dense_218 (Dense)            (None, 100)               30100     
_________________________________________________________________
dense_219 (Dense)            (None, 50)                5050      
_________________________________________________________________
dense_220 (Dense)            (None, 50)                2550      
_________________________________________________________________
dense_221 (Dense)            (None, 50)                2550      
_________________________________________________________________
dense_222 (Dense)            (None, 1)                

In [40]:
model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

In [41]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [42]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(lr=1e-3),
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

Train on 200 samples, validate on 986 samples
Epoch 1/4
200/200 [==============================] - 1s 4ms/sample - loss: 0.5512 - accuracy: 0.7150 - val_loss: 0.5864 - val_accuracy: 0.6694
Epoch 2/4
200/200 [==============================] - 0s 941us/sample - loss: 0.5347 - accuracy: 0.7200 - val_loss: 0.5710 - val_accuracy: 0.6775
Epoch 3/4
200/200 [==============================] - 0s 1ms/sample - loss: 0.5196 - accuracy: 0.7450 - val_loss: 0.5567 - val_accuracy: 0.6897
Epoch 4/4
200/200 [==============================] - 0s 987us/sample - loss: 0.5055 - accuracy: 0.7550 - val_loss: 0.5426 - val_accuracy: 0.7039


In [43]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(lr=1e-3),
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Train on 200 samples, validate on 986 samples
Epoch 1/16
200/200 [==============================] - 1s 6ms/sample - loss: 0.4368 - accuracy: 0.8000 - val_loss: 0.3965 - val_accuracy: 0.8418
Epoch 2/16
200/200 [==============================] - 0s 1ms/sample - loss: 0.3236 - accuracy: 0.8850 - val_loss: 0.3199 - val_accuracy: 0.8884
Epoch 3/16
200/200 [==============================] - 0s 1ms/sample - loss: 0.2601 - accuracy: 0.9400 - val_loss: 0.2709 - val_accuracy: 0.9158
Epoch 4/16
200/200 [==============================] - 0s 1ms/sample - loss: 0.2177 - accuracy: 0.9650 - val_loss: 0.2321 - val_accuracy: 0.9402
Epoch 5/16
200/200 [==============================] - 0s 1ms/sample - loss: 0.1847 - accuracy: 0.9850 - val_loss: 0.2052 - val_accuracy: 0.9544
Epoch 6/16
200/200 [==============================] - 0s 1ms/sample - loss: 0.1616 - accuracy: 0.9850 - val_loss: 0.1836 - val_accuracy: 0.9625
Epoch 7/16
200/200 [==============================] - 0s 1ms/sample - loss: 0.1430 - accur

In [44]:
model_B.evaluate(X_test_B, y_test_B)

2000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.14446661782264708, 0.9695]

In [45]:
model_B_on_A.evaluate(X_test_B, y_test_B)

2000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.09330978256464005, 0.99]

So, the accuracy increased from 0.9695 to 0.99!!

# Exercise

a.Build a DNN with 20 hidden layers of 100 neurons each (that's too many, but it's the point of this exercise). Use He initialization and the ELU activation function.

In [46]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32,32,3]))
for layer in range(20):
    model.add(keras.layers.Dense(100,activation="elu",
                                kernel_initializer="he_normal"))

b.Using Nadam optimization and early stopping, train the network on the CIFAR10 dataset. You can load it with keras.datasets.cifar10.load_data(). The dataset is composed of 60,000 32 × 32–pixel color images (50,000 for training, 10,000 for testing) with 10 classes, so you'll need a softmax output layer with 10 neurons. Remember to search for the right learning rate each time you change the model's architecture or hyperparameters.

Load the dataset

In [47]:
(X_train_full,y_train_full),(X_test,y_test) = keras.datasets.cifar10.load_data()

X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

Create output layer

In [48]:
model.add(keras.layers.Dense(10,activation="softmax"))

Now using Nadam Optimizer. Learning rate 5e-5 gives better result so it has been used

In [49]:
optimizer = keras.optimizers.Nadam(lr=5e-5)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

Now create a callback to train the model

In [50]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [51]:
%reload_ext tensorboard
%tensorboard --logdir=./my_logs --port=6006

Reusing TensorBoard on port 6006 (pid 27597), started 1 day, 21:54:31 ago. (Use '!kill 27597' to kill it.)

In [52]:
model.fit(X_train, y_train, epochs=10,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

Train on 45000 samples, validate on 5000 samples
Epoch 1/10
45000/45000 [==============================] - 52s 1ms/sample - loss: 4.2440 - accuracy: 0.1559 - val_loss: 2.1511 - val_accuracy: 0.2154
Epoch 2/10
45000/45000 [==============================] - 46s 1ms/sample - loss: 2.0626 - accuracy: 0.2398 - val_loss: 1.9661 - val_accuracy: 0.2758
Epoch 3/10
45000/45000 [==============================] - 46s 1ms/sample - loss: 1.9387 - accuracy: 0.2865 - val_loss: 1.9005 - val_accuracy: 0.3002
Epoch 4/10
45000/45000 [==============================] - 45s 999us/sample - loss: 1.8670 - accuracy: 0.3213 - val_loss: 1.8214 - val_accuracy: 0.3390
Epoch 5/10
45000/45000 [==============================] - 45s 993us/sample - loss: 1.8045 - accuracy: 0.3450 - val_loss: 1.9185 - val_accuracy: 0.3142
Epoch 6/10
45000/45000 [==============================] - 46s 1ms/sample - loss: 1.7603 - accuracy: 0.3603 - val_loss: 1.7956 - val_accuracy: 0.3526
Epoch 7/10
45000/45000 [=============================

In [53]:
model = keras.models.load_model("my_cifar10_model.h5")
model.evaluate(X_valid, y_valid)

5000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[1.6639602180480957, 0.4036]

c.Now try adding Batch Normalization and compare the learning curves: Is it converging faster than before? Does it produce a better model? How does it affect training speed?

In [56]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32,32,3]))
model.add(keras.layers.BatchNormalization())
for layer in range(20):
    model.add(keras.layers.Dense(100,
                                kernel_initializer="he_normal"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("elu"))
model.add(keras.layers.Dense(10,activation="softmax"))

optimizer = keras.optimizers.Nadam(lr=5e-5)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_bn_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_bn_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

model.fit(X_train, y_train, epochs=10,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

model = keras.models.load_model("my_cifar10_bn_model.h5")
model.evaluate(X_valid, y_valid)

Train on 45000 samples, validate on 5000 samples
Epoch 1/10
45000/45000 [==============================] - 92s 2ms/sample - loss: 2.0412 - accuracy: 0.2684 - val_loss: 1.7709 - val_accuracy: 0.3652
Epoch 2/10
45000/45000 [==============================] - 79s 2ms/sample - loss: 1.7806 - accuracy: 0.3644 - val_loss: 1.6525 - val_accuracy: 0.4030
Epoch 3/10
45000/45000 [==============================] - 81s 2ms/sample - loss: 1.6887 - accuracy: 0.3974 - val_loss: 1.5883 - val_accuracy: 0.4320
Epoch 4/10
45000/45000 [==============================] - 79s 2ms/sample - loss: 1.6317 - accuracy: 0.4188 - val_loss: 1.5369 - val_accuracy: 0.4474
Epoch 5/10
45000/45000 [==============================] - 80s 2ms/sample - loss: 1.5843 - accuracy: 0.4358 - val_loss: 1.5203 - val_accuracy: 0.4546
Epoch 6/10
45000/45000 [==============================] - 81s 2ms/sample - loss: 1.5477 - accuracy: 0.4487 - val_loss: 1.4835 - val_accuracy: 0.4686
Epoch 7/10
45000/45000 [==============================] -

[1.4189116363525391, 0.498]

d.Try replacing Batch Normalization with SELU, and make the necessary adjustements to ensure the network self-normalizes (i.e., standardize the input features, use LeCun normal initialization, make sure the DNN contains only a sequence of dense layers, etc.).

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32,32,3]))
model.add(keras.layers.BatchNormalization())
for layer in range(20):
    model.add(keras.layers.Dense(100,
                                kernel_initializer="lecun_normal",
                                activation="selu"))
model.add(keras.layers.Dense(10,activation="softmax"))

optimizer = keras.optimizers.Nadam(lr=5e-5)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_selu_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_selu_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

X_means = X_train.mean(axis=0, keepdims=True)
X_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

model.fit(X_train_scaled, y_train, epochs=10,
          validation_data=(X_valid_scaled, y_valid),
          callbacks=callbacks)

model = keras.models.load_model("my_cifar10_selu_model.h5")
model.evaluate(X_valid, y_valid)